<a href="https://colab.research.google.com/github/KhanhPham2411/Reinforcement-Learning-Gold-Miner/blob/master/scripts/TrainingClient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/My Drive/Sync/Reinforcement-Learning-Gold-Miner

/content/drive/My Drive/Sync/Reinforcement-Learning-Gold-Miner


In [2]:
import sys
sys.path.append("src")

In [3]:
from MinerGymEnv import MinerGymEnv
from Model.DQNModel import DQN # A class of creating a deep q-learning model
from MinerEnv import MinerEnv # A class of creating a communication environment between the DQN model and the GameMiner environment (GAME_SOCKET_DUMMY.py)
from Memory import Memory # A class of creating a batch in order to store experiences for the training process

import pandas as pd
import datetime 
import numpy as np

In [4]:
from keras import backend as K
K.set_session

Using TensorFlow backend.


<function keras.backend.tensorflow_backend.set_session>

In [5]:
HOST = "localhost"
PORT = 1111


# Create header for saving DQN learning file
now = datetime.datetime.now() #Getting the latest datetime
header = ["Ep", "Step", "Reward", "Total_reward", "Actdion", "Epsilon", "Done", "Termination_Code"] #Defining header for the save file
filename = "Data/data_" + now.strftime("%Y%m%d-%H%M") + ".csv" 
with open(filename, 'w') as f:
    pd.DataFrame(columns=header).to_csv(f, encoding='utf-8', index=False, header=True)

# Parameters for training a DQN model
N_EPISODE = 10000 #The number of episodes for training
MAX_STEP = 1000   #The number of steps for each episode
BATCH_SIZE = 32   #The number of experiences for each replay 
MEMORY_SIZE = 100000 #The size of the batch for storing experiences
SAVE_NETWORK = 100  # After this number of episodes, the DQN model is saved for testing later. 
INITIAL_REPLAY_SIZE = 1000 #The number of experiences are stored in the memory batch before starting replaying
INPUTNUM = 209 #The number of input values for the DQN model
ACTIONNUM = 6  #The number of actions output from the DQN model
MAP_MAX_X = 21 #Width of the Map
MAP_MAX_Y = 9  #Height of the Map
DEBUG = False

In [ ]:
# Initialize a DQN model and a memory batch for storing experiences
DQNAgent = DQN(INPUTNUM, ACTIONNUM)
memory = Memory(MEMORY_SIZE)

# Initialize environment
minerEnv = MinerGymEnv(HOST, PORT,debug=DEBUG) #Creating a communication environment between the DQN model and the game environment (GAME_SOCKET_DUMMY.py)
minerEnv.start()  # Connect to the game

train = False #The variable is used to indicate that the replay starts, and the epsilon starts decrease.
#Training Process
#the main part of the deep-q learning agorithm 


In [9]:
ls

'code sample'/   init_docker_env.sh   requirements.txt   TrainedModels/
 Data/           Maps/                scripts/
 docs/           README.md            src/


In [14]:
DQNAgent.model.load_weights('TrainedModels/DQNmodel_20200726-1939_ep2200.h5')
DQNAgent.target_model.load_weights('TrainedModels/DQNmodel_20200726-1939_ep2200.h5')

minerEnv.reset() 
s = minerEnv.get_state()
                        
total_reward = 0
terminate = False 
maxStep = minerEnv.state.mapInfo.maxStep 

for step in range(0, maxStep):
    # action = DQNAgent.act(s)
    q_predict = DQNAgent.model.predict(s.reshape(1,len(s)))
    action = np.argmax(q_predict)  
    
    minerEnv.step(str(action))
    s_next = minerEnv.get_state()  
    reward = minerEnv.get_reward() 
    terminate = minerEnv.check_terminate()  

    total_reward = total_reward + reward #Plus the reward to the total rewad of the episode
    s = s_next #Assign the next state for the next step.

    print(f'steps:{step} action:{action} q_predict:{q_predict}')
    if terminate == True:
        #If the episode ends, then go to the next episode
        break
#Print the training information after the episode
print('Number of steps is: %d. Accumulated Reward = %.2f. Epsilon = %.2f .Termination code: %d' % (
    step + 1, total_reward, DQNAgent.epsilon, terminate))

steps:0 action:3 q_predict:[[14.337856   3.2386677 19.819118  29.563423  22.053093   5.3174124]]
steps:1 action:3 q_predict:[[14.244743   3.6381357 19.84628   28.574999  20.406996   3.4428375]]
steps:2 action:3 q_predict:[[13.796219   3.2086146 19.084955  27.717543  20.150787   3.762162 ]]
steps:3 action:3 q_predict:[[13.498821   3.0011873 18.680761  27.368818  19.989891   3.8350916]]
steps:4 action:3 q_predict:[[11.376065   2.7833967 17.033302  24.538347  17.530657   2.4213662]]
steps:5 action:3 q_predict:[[ 8.71283    2.1607308 15.425867  20.71021   14.70153   -1.2131127]]
steps:6 action:3 q_predict:[[ 7.856098   1.3043858 15.008596  19.247156  13.95612   -1.1949588]]
steps:7 action:3 q_predict:[[ 7.3578405  1.1467979 14.966654  18.927147  13.856074  -1.7681893]]
steps:8 action:3 q_predict:[[ 6.9282     0.8308905 14.747862  18.355257  13.491598  -2.1669035]]
steps:9 action:3 q_predict:[[ 5.3715963  -0.11215611 14.029888   16.161106   11.920361   -4.231462  ]]
Number of steps is: 10. 

In [7]:
return

for episode_i in range(0, N_EPISODE):
    try:
        # Choosing a map in the list
        mapID = np.random.randint(1, 6) #Choosing a map ID from 5 maps in Maps folder randomly
        posID_x = np.random.randint(MAP_MAX_X) #Choosing a initial position of the DQN agent on X-axes randomly
        posID_y = np.random.randint(MAP_MAX_Y) #Choosing a initial position of the DQN agent on Y-axes randomly
        #Creating a request for initializing a map, initial position, the initial energy, and the maximum number of steps of the DQN agent
        request = ("map" + str(mapID) + "," + str(posID_x) + "," + str(posID_y) + ",50,100") 
        #Send the request to the game environment (GAME_SOCKET_DUMMY.py)
        minerEnv.send_map_info(request)

        # Getting the initial state
        minerEnv.reset() #Initialize the game environment
        s = minerEnv.get_state()#Get the state after reseting. 
                                #This function (get_state()) is an example of creating a state for the DQN model 
        total_reward = 0 #The amount of rewards for the entire episode
        terminate = False #The variable indicates that the episode ends
        maxStep = minerEnv.state.mapInfo.maxStep #Get the maximum number of steps for each episode in training
        #Start an episde for training
        for step in range(0, maxStep):
            action = DQNAgent.act(s)  # Getting an action from the DQN model from the state (s)
            minerEnv.step(str(action))  # Performing the action in order to obtain the new state
            s_next = minerEnv.get_state()  # Getting a new state
            reward = minerEnv.get_reward()  # Getting a reward
            terminate = minerEnv.check_terminate()  # Checking the end status of the episode

            # Add this transition to the memory batch
            memory.push(s, action, reward, terminate, s_next)

            # Sample batch memory to train network
            if (memory.length > INITIAL_REPLAY_SIZE):
                #If there are INITIAL_REPLAY_SIZE experiences in the memory batch
                #then start replaying
                batch = memory.sample(BATCH_SIZE) #Get a BATCH_SIZE experiences for replaying
                DQNAgent.replay(batch, BATCH_SIZE)#Do relaying
                train = True #Indicate the training starts
            total_reward = total_reward + reward #Plus the reward to the total rewad of the episode
            s = s_next #Assign the next state for the next step.

            # Saving data to file
            save_data = np.hstack(
                [episode_i + 1, step + 1, reward, total_reward, action, DQNAgent.epsilon, terminate]).reshape(1, 7)
            with open(filename, 'a') as f:
                pd.DataFrame(save_data).to_csv(f, encoding='utf-8', index=False, header=False)
            
            if terminate == True:
                #If the episode ends, then go to the next episode
                break

        # Iteration to save the network architecture and weights
        if (np.mod(episode_i + 1, SAVE_NETWORK) == 0 and train == True):
            DQNAgent.target_train()  # Replace the learning weights for target model with soft replacement
            #Save the DQN model
            now = datetime.datetime.now() #Get the latest datetime
            DQNAgent.save_model("TrainedModels/",
                                "DQNmodel_" + now.strftime("%Y%m%d-%H%M") + "_ep" + str(episode_i + 1))



        #Print the training information after the episode
        print('Episode %d ends. Number of steps is: %d. Accumulated Reward = %.2f. Epsilon = %.2f .Termination code: %d' % (
            episode_i + 1, step + 1, total_reward, DQNAgent.epsilon, terminate))
        
        #Decreasing the epsilon if the replay starts
        if train == True:
            DQNAgent.update_epsilon()

    except Exception as e:
        import traceback

        traceback.print_exc()
        # print("Finished.")
        break

SyntaxError: ignored